# Importing Data and libraries

In [26]:
import pandas as pd
import numpy as np
import calendar
import seaborn as sns
import matplotlib.pyplot as plt


In [27]:
df = pd.read_excel(r'C:\Users\USER\Downloads\covid-19-state-level-data.xlsx')

In [28]:
df.drop('Unnamed: 0',axis = 1,inplace= True)

In [29]:
df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8154 entries, 0 to 8153
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    8154 non-null   datetime64[ns]
 1   state   8154 non-null   object        
 2   fips    8154 non-null   int64         
 3   cases   8154 non-null   int64         
 4   deaths  8154 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 318.6+ KB


In [31]:
df.isna().sum()

date      0
state     0
fips      0
cases     0
deaths    0
dtype: int64

In [32]:
print(df['date'].min())
print(df['date'].max())

2020-01-21 00:00:00
2020-07-28 00:00:00


In [33]:
df['month'] = df['date'].dt.month

## we need month of march,april,may and june --> 3,4,5,6

In [34]:
df_new = df[df['month'].isin([3,4,5,6])]

In [35]:
df_new.head()

,date,state,fips,cases,deaths,month
240,2020-03-01,Arizona,4,1,0,3
241,2020-03-01,California,6,33,0,3
242,2020-03-01,Florida,12,2,0,3
243,2020-03-01,Illinois,17,3,0,3
244,2020-03-01,Massachusetts,25,1,0,3


In [36]:
df_new.shape

(6374, 6)

In [37]:
df_new['month'].unique()

array([3, 4, 5, 6], dtype=int64)

In [44]:
res = df_new.groupby(['month','state'])['deaths'].sum()

In [57]:
result_df = res.reset_index().groupby('month').apply(lambda x: x.nlargest(3, 'deaths')).reset_index(drop = True)

In [58]:
result_df

,month,state,deaths
0,3,New York,7943
1,3,Washington,2377
2,3,New Jersey,1165
3,4,New York,425198
4,4,New Jersey,102708
5,4,Michigan,59519
6,5,New York,854088
7,5,New Jersey,308935
8,5,Massachusetts,170827
9,6,New York,918476


## Total deaths respective to month

In [83]:
total_deaths = df_new.groupby('month')['deaths'].sum().to_dict()

In [89]:
total_deaths

{3: 20322, 4: 990495, 5: 2695260, 6: 3498953}

In [87]:
result_df['%_of_tot'] = result_df.apply(lambda x : np.round(x['deaths']/total_deaths[x['month']],2),axis = 1 )

In [88]:
result_df

,month,state,deaths,%_of_tot
0,3,New York,7943,0.39
1,3,Washington,2377,0.12
2,3,New Jersey,1165,0.06
3,4,New York,425198,0.43
4,4,New Jersey,102708,0.10
5,4,Michigan,59519,0.06
6,5,New York,854088,0.32
7,5,New Jersey,308935,0.11
8,5,Massachusetts,170827,0.06
9,6,New York,918476,0.26


## Printing the top 3 states based on number of deaths on the month

In [106]:
for month,summary in result_df.groupby('month'):
    print(
f'''Month - {calendar.month_name[month]}
{summary['state'].iloc[0]} - {summary['deaths'].iloc[0]}, {summary['%_of_tot'].iloc[0]}%
{summary['state'].iloc[1]} - {summary['deaths'].iloc[1]}, {summary['%_of_tot'].iloc[1]}%
{summary['state'].iloc[2]} - {summary['deaths'].iloc[2]}, {summary['%_of_tot'].iloc[2]}%'''
    )


Month - March
New York - 7943, 0.39%
Washington - 2377, 0.12%
New Jersey - 1165, 0.06%
Month - April
New York - 425198, 0.43%
New Jersey - 102708, 0.1%
Michigan - 59519, 0.06%
Month - May
New York - 854088, 0.32%
New Jersey - 308935, 0.11%
Massachusetts - 170827, 0.06%
Month - June
New York - 918476, 0.26%
New Jersey - 388821, 0.11%
Massachusetts - 228975, 0.07%
